To set up the enviroment I used <br>
conda create -n ines python=3.12 <br>
Then I ran <br>
pip3 install torch torchvision torchaudio <br>
Then I ran <br>
conda install matplotlib <br>
conda install pandas <br>
The data was downloaded from <br>
https://archive.ics.uci.edu/dataset/186/wine+quality

In [1]:
# Our imports
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
#import matplotlib.pyplot as plt
#import torch.nn.functional as F
import torch.utils.data as data_utils
from torch.utils.data import DataLoader, TensorDataset, random_split
#from torch.utils.tensorboard import SummaryWriter
#import time

In [2]:
# This doesn't work as the first line is not a csv format
mydataNump = np.genfromtxt('winequality-red.csv', delimiter=';', skip_header=2)
print(mydataNump)

[[ 7.8    0.88   0.    ...  0.68   9.8    5.   ]
 [ 7.8    0.76   0.04  ...  0.65   9.8    5.   ]
 [11.2    0.28   0.56  ...  0.58   9.8    6.   ]
 ...
 [ 6.3    0.51   0.13  ...  0.75  11.     6.   ]
 [ 5.9    0.645  0.12  ...  0.71  10.2    5.   ]
 [ 6.     0.31   0.47  ...  0.66  11.     6.   ]]


In [3]:
mydatapd = pd.read_csv('winequality-red.csv', delimiter=';')
mydatapd.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


In [4]:
print("The original data is this shape",mydataNump.shape)
winedata = mydataNump[:-100, 0:11]

print("I cut out the growing data and leave 100 rows for later giving", winedata.shape)

quality = mydataNump[:-100:, 11]
print("finally I slect all but the last 100 quaity values", quality.shape)

The original data is this shape (1598, 12)
I cut out the growing data and leave 100 rows for later giving (1498, 11)
finally I slect all but the last 100 quaity values (1498,)


In [5]:
class mynetwork(nn.Module):
    def __init__(self, middle_width, drop_out):
        super().__init__()
        self.linear = nn.Linear(11, int(middle_width/2))
        self.m = nn.Sigmoid()
        self.r = nn.ReLU()
        self.s = nn.Softmax(dim=1)
        self.linear1 = nn.Linear(int(middle_width/2), middle_width)
        self.linear2 = nn.Linear(middle_width, int(middle_width/2))
        self.linear3 = nn.Linear(int(middle_width/2), 1)
        self.drop = nn.Dropout(drop_out)

    def forward(self, xb):
        xb = self.linear(xb)
        xb = self.r(xb)
        xb = self.linear1(xb)
        xb = self.drop(xb)
        xb = self.r(xb)
        xb = self.linear2(xb)
        xb = self.r(xb)
        out = self.linear3(xb)
        #out = self.s(xb)
        return out

In [6]:
wineData_tensor = torch.from_numpy(winedata).float()
quality_tensor = torch.from_numpy(quality).unsqueeze(1).float()
train_tensor = data_utils.TensorDataset(wineData_tensor, quality_tensor)
loader = DataLoader(train_tensor, batch_size=50, shuffle=True)
first_batch = next(iter(loader))
characteristics, labels = first_batch


In [7]:
print(first_batch[0].shape, first_batch[0].type())
print(first_batch[1].shape, first_batch[1].type())

torch.Size([50, 11]) torch.FloatTensor
torch.Size([50, 1]) torch.FloatTensor


In [8]:
network = mynetwork(100, 0.1)

In [9]:
preds = network(characteristics)

In [10]:
print(preds.shape)
print(preds)

torch.Size([50, 1])
tensor([[ 0.0203],
        [ 0.0161],
        [ 0.0646],
        [-0.1890],
        [-0.3137],
        [ 0.1093],
        [ 0.0223],
        [-0.3674],
        [-0.1224],
        [-0.0368],
        [-0.1023],
        [-0.1554],
        [ 0.0697],
        [ 0.0819],
        [ 0.1583],
        [-0.0313],
        [ 0.1405],
        [-0.1580],
        [-0.3177],
        [-0.0318],
        [ 0.0934],
        [ 0.0201],
        [ 0.0152],
        [-0.1176],
        [ 0.0452],
        [ 0.1334],
        [-0.1216],
        [ 0.1106],
        [ 0.0199],
        [ 0.1177],
        [ 0.0493],
        [ 0.0160],
        [-0.0091],
        [-0.0096],
        [-0.3414],
        [ 0.1070],
        [ 0.0616],
        [-0.2374],
        [-0.0874],
        [-0.2508],
        [-0.0164],
        [-0.5199],
        [-0.4052],
        [-1.2882],
        [ 0.0963],
        [ 0.0555],
        [ 0.1769],
        [ 0.1064],
        [ 0.0414],
        [-0.1434]], grad_fn=<AddmmBackward0>)
